## Uniprot API to extract Protein Data

In [1]:
#!pip install xmlschema

### [Programmatic access - Retrieving entries via queries](https://www.uniprot.org/help/api_queries)


In [1]:
"""
Queries Uniprot database and retrieves protein data.
"""
import urllib.request
from urllib.error import HTTPError
from lxml import etree
import lxml.etree
import lxml.html
import json
import xmlschema
import lxml
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd

# FUNCTIONS

In [82]:
def unroll(data, master_list, ec):
    if type(data) == dict:
        for key, value in data.items():
            if(key == 'ecNumber'):
                ec = True
            unroll(data[key], master_list, ec)  
    elif type(data) == list:
        for i in range(0,len(data)):
            unroll(data[i], master_list, ec)
    else:
        if(type(data) != int):
            if(ec == True):
                data = "EC " + data
            master_list.append(data)

def findNewIDforOldID(oldID):
    print('Looking for the oldID', oldID)
    source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
    webpage = BeautifulSoup(source, 'html')
    main = webpage.find('main')
    altIDsSection = main.find_all('div', class_= 'entryEmpty')
    altIDs = []
    for each_ID in altIDsSection:
        links = each_ID.find_all('a', href=True)
        for link in links:
            altIDlink = link['href'][9:]   #removes prefix of link containing ID
            altIDs.append(altIDlink)
        
    #Remove non-protein IDs from altIDs
    endOfaltIDs = 0
    for index in range(0, len(altIDs)):
        if('replaces' in altIDs[index]):
            endOfaltIDs = index
            break
    realAltIDs = []
    for index in range(0, endOfaltIDs):
        realAltIDs.append(altIDs[index])
    return(realAltIDs)

def findNewIDforOldID_SecondAttempt(oldID):
    #FIRST: Finding the link to the next page /html/body/main/div/p/a[1]
    source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
    webpage = BeautifulSoup(source, 'html')
    altID_Link = 'https://www.uniprot.org'+webpage.find('main').find('div', class_= 'entryEmpty').find("p").find("a")['href']
    
    print('The protein ID will be looked for at this link: ', altID_Link)

    #SECOND: Using the link for the next page to find the existing ID: /html/body/main/div/div[3]/table/tbody/tr/td[4]/a[2
                                                                    #  /html/body/main/div/div[3]/div[4]/table/tbody/tr[1]/td[2]/div/div]
    source2 = requests.get(altID_Link).text
    webpage2 = BeautifulSoup(source2, 'html')
    main = webpage2.find('main',class_='uniparc') #.find('div', class_='content results').find('table', class_='grid')#.find('tbody')
    step1 = main.find('div',class_='main').find('div',class_='content results').find('table').find('tbody').find('tr').find_all('td')
    thePossibleIDs = step1[3].find_all('a')
    thePossibleIDs_List = []
    for ID_Link in thePossibleIDs:
        thePossibleIDs_List.append(ID_Link['href'][9:])
    #Finds the ID(s) out of the final options from the last section
    theRightIDs = []
    for index in range(0, len(thePossibleIDs_List)):
        if('version' not in thePossibleIDs_List[index]):
            theRightIDs.append(thePossibleIDs_List[index])
    return(theRightIDs)

def findURLxml(ID):
    unrolledSynList = []
    url = 'https://www.uniprot.org/uniprot/'+ID+'.xml'
    with urllib.request.urlopen(url) as r:                     
        raw_data = r.read().strip()
    tree = etree.fromstring(raw_data); 
    entry_dict = schema.to_dict(tree);
    content = entry_dict['entry'][0]
    try: species_name = content['organism']['name'][0]['$']
    except: species_name = ''; print('Error with species name for ', ID)
    unroll(content['protein'], unrolledSynList, False)
    return unrolledSynList, species_name



def findURL(ID):
    synList = []; species_name = 'N/A'; stillMissIDs = []; foundidList = []; notfoundidList = [];
    try: 
        subSynList, species_name = findURLxml(ID); print('1st Try')
        synList.append(subSynList)
        foundidList.append(ID)
    except: 
        try: 
            altIDs = findNewIDforOldID(ID) #To find synonyms. If found on 2nd try. There might be multiple new IDs
            for altID in altIDs:
                try: 
                    subSynList = []
                    syns, sn = findURLxml(altID); 
                    for s in syns: subSynList.append(s)
                    synList.append(subSynList)
                    foundidList.append(altID)
                except: stillMissIDs.append(altID)
            print('2nd try');
        except: stillMissIDs.append(ID) #If there weren't any new IDs
        if(len(stillMissIDs) > 0): #If there are still IDs without an entry
            for ID in stillMissIDs:
                try: 
                    altIDs2 = findNewIDforOldID_SecondAttempt(ID); print('3rd try')
                    for altID2 in altIDs2:
                        try:
                            subSynList = []
                            syns, sn = findURLxml(altID2) 
                            for s in syns: synList.append(s)
                            synList.append(subSynList)
                            foundidList.append(altID2)
                        except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(altID2)
                except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(ID)
    allIDs = []
    for n in range(0,len(foundidList)): allIDs.append(foundidList[n])
    for n in range(0,len(notfoundidList)): allIDs.append(notfoundidList[n])
    
    return allIDs, notfoundidList, foundidList, synList, species_name

def findURL(ID):
    synList = []; species_name = 'N/A'; stillMissIDs = []; foundidList = []; notfoundidList = [];
    try: 
        subSynList, species_name = findURLxml(ID); print('1st Try')
        synList.append(subSynList)
        foundidList.append(ID)
    except: 
        try: 
            altIDs = findNewIDforOldID(ID) #To find synonyms. If found on 2nd try. There might be multiple new IDs
            for altID in altIDs:
                try: 
                    subSynList = []
                    syns, sn = findURLxml(altID); 
                    for s in syns: subSynList.append(s)
                    synList.append(subSynList)
                    foundidList.append(altID)
                except: stillMissIDs.append(altID)
            print('2nd try');
        except: stillMissIDs.append(ID) #If there weren't any new IDs
        if(len(stillMissIDs) > 0): #If there are still IDs without an entry
            for ID in stillMissIDs:
                try: 
                    altIDs2 = findNewIDforOldID_SecondAttempt(ID); print('3rd try')
                    for altID2 in altIDs2:
                        try:
                            subSynList = []
                            syns, sn = findURLxml(altID2) 
                            for s in syns: synList.append(s)
                            synList.append(subSynList)
                            foundidList.append(altID2)
                        except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(altID2)
                except(HTTPError, lxml.etree.XMLSyntaxError) as err: print('Missing:', altID2); notfoundidList.append(ID)
    allIDs = []
    for n in range(0,len(foundidList)): allIDs.append(foundidList[n])
    for n in range(0,len(notfoundidList)): allIDs.append(notfoundidList[n])
    
    return allIDs, notfoundidList, foundidList, synList, species_name

# DEMONSTRATING FUNCTIONS

### Demonstrating Query to search a single Protein detail

In [83]:
protein = 'P08574'
format = '.xml'
url = 'https://www.uniprot.org/uniprot/'+protein+ format   #The URL with the protein.xml at the end
        
with urllib.request.urlopen(url) as r:                     #Reads the data from the URL for the particular protein
    raw_data = r.read().strip()

tree = etree.fromstring(raw_data)                          #Makes a tree out of the protein's raw data from UniProt
schema = xmlschema.XMLSchema('https://www.uniprot.org/docs/uniprot.xsd') #Makes a schema out of the XML structure
entry_dict = schema.to_dict(tree)       #Makes a dictionary out of the XML schema. (What's the 'tree' argument?)
content = entry_dict['entry'][0]        #The first entry of the dictionary is assigned a variable. It has the xml structure of the protein info
names = content['protein']                      #The information about the protein and it's other info is displayed here and below

In [84]:
names

{'alternativeName': [{'fullName': 'Complex III subunit 4'},
  {'fullName': 'Complex III subunit IV'},
  {'fullName': 'Cytochrome b-c1 complex subunit 4'},
  {'fullName': 'Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit',
   'shortName': ['Cytochrome c-1']}],
 'recommendedName': {'ecNumber': ['7.1.1.8'],
  'fullName': 'Cytochrome c1, heme protein, mitochondrial'}}

In [85]:
ec=False
unrolledNamesTest = []
unroll(names, unrolledNamesTest, ec)
print(unrolledNamesTest)

['Cytochrome c1, heme protein, mitochondrial', 'EC 7.1.1.8', 'Complex III subunit 4', 'Complex III subunit IV', 'Cytochrome b-c1 complex subunit 4', 'Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit', 'Cytochrome c-1']


### Demonstrating Old ID Replacement Function 

In [86]:
oldID = 'P30042'
source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
webpage = BeautifulSoup(source, 'html')
main = webpage.find('main')
altIDsSection = main.find_all('div', class_= 'entryEmpty')
altIDs = []
for each_ID in altIDsSection:
    links = each_ID.find_all('a', href=True)
    for link in links:
        altIDlink = link['href'][9:]   #removes prefix of link containing ID
        altIDs.append(altIDlink)
        
#Remove non-protein IDs from altIDs
for index in range(0, len(altIDs)):
    if('replaces' in altIDs[index]):
        endOfaltIDs = index
        break
realAltIDs = []
for index in range(0, endOfaltIDs):
    realAltIDs.append(altIDs[index])
print(realAltIDs)

['A0A0B4J2D5', 'P0DPI2']


### Demonstrating Second Attempt Function

In [87]:
#def findNewIDforOldID_SecondAttempt(oldID)

#Finding the link to the next page /html/body/main/div/p/a[1]
oldID = 'Q9H2Q1'
source = requests.get('https://www.uniprot.org/uniprot/'+oldID+ '.html').text
webpage = BeautifulSoup(source, 'html')
altID_Link = 'https://www.uniprot.org'+webpage.find('main').find('div', class_= 'entryEmpty').find("p").find("a")['href']
print(altID_Link)

#Using the link for the next page to find the existing ID: /html/body/main/div/div[3]/table/tbody/tr/td[4]/a[2]
source2 = requests.get(altID_Link).text
webpage2 = BeautifulSoup(source2, 'html')
main = webpage2.find('main',class_='uniparc') #.find('div', class_='content results').find('table', class_='grid')#.find('tbody')
step1 = main.find('div',class_='main').find('div',class_='content results').find('table').find('tbody').find('tr').find_all('td')
thePossibleIDs = step1[3].find_all('a')
thePossibleIDs_List = []
for ID_Link in thePossibleIDs:
    thePossibleIDs_List.append(ID_Link['href'][9:])

theRightIDs = []
for index in range(0, len(thePossibleIDs_List)):
    if('version' not in thePossibleIDs_List[index]):
        theRightIDs.append(thePossibleIDs_List[index])
print(theRightIDs)


https://www.uniprot.org/uniparc/?query=Q9H2Q1
['A0A499FIA2']


# MAIN PROGRAM: COLLECTING PROTEIN NAMES

In [356]:
# If using a different file, switch out the file name here
df = pd.read_csv("Human.MitoCarta3.0.csv")
df.tail(5)

,HumanGeneID,MouseOrthologGeneID,Symbol,Synonyms,Description,MitoCarta3.0_List,MitoCarta3.0_Evidence,MitoCarta3.0_SubMitoLocalization,MitoCarta3.0_MitoPathways,TrainingDataset,...,liver_total_peak_intensity_log10,heart_total_peak_intensity_log10,skeletalmuscle_total_peak_intensity_log10,adipose_total_peak_intensity_log10,smallintestine_total_peak_intensity_log10,largeintestine_total_peak_intensity_log10,stomach_total_peak_intensity_log10,placenta_total_peak_intensity_log10,testis_total_peak_intensity_log10,HPA_Main_Location_2020 (Reliability)
1131,339983,269642.0,NAT8L,CML3|NACED|NAT8-LIKE,N-acetyltransferase 8 like,MitoCarta3.0,literature,Membrane,Metabolism > Amino acid metabolism > Glutamate...,Tpossible_mito,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mitochondria (Approved)
1132,401250,NaN,MCCD1,-,mitochondrial coiled-coil domain 1,MitoCarta3.0,manual,unknown,0,Tpossible_mito,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1133,101928527,66364.0,PIGBOS1,HP06981,PIGB opposite strand 1,MitoCarta3.0,literature,MOM,Mitochondrial dynamics and surveillance > Orga...,NA - newly added to NCBI Entrez Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1134,109703458,109729085.0,HTD2,-,hydroxyacyl-thioester dehydratase type 2,MitoCarta3.0,literature,Matrix,Metabolism > Lipid metabolism > Type II fatty ...,NA - newly added to NCBI Entrez Gene,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mitochondria (Supported)
1135,ENSG00000230623,NaN,RP11_469A15.2,-,RP11_469A15.2,MitoCarta3.0,literature,MIM,NaN,NA - newly define mitochondrial protein (forme...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [357]:
uids = list(df['UniProt'])
uids[0:5]

['P08574', 'P21912', 'Q99807', 'P31040', 'P31930']

In [358]:
# Create dataframe which will store UniProt IDs and Synonyms
df2 = pd.DataFrame(columns = ["UniProt ID", "Synonyms"])
for i in range(0,len(df)):
    df2 = df2.append({"UniProt ID": uids[i], "Synonyms": []}, ignore_index=True)
lendf2 = len(df2)

### Find and Add Synonyms from UniProt and MitoCarta


In [359]:
df[1135:]

,HumanGeneID,MouseOrthologGeneID,Symbol,Synonyms,Description,MitoCarta3.0_List,MitoCarta3.0_Evidence,MitoCarta3.0_SubMitoLocalization,MitoCarta3.0_MitoPathways,TrainingDataset,...,liver_total_peak_intensity_log10,heart_total_peak_intensity_log10,skeletalmuscle_total_peak_intensity_log10,adipose_total_peak_intensity_log10,smallintestine_total_peak_intensity_log10,largeintestine_total_peak_intensity_log10,stomach_total_peak_intensity_log10,placenta_total_peak_intensity_log10,testis_total_peak_intensity_log10,HPA_Main_Location_2020 (Reliability)
1135,ENSG00000230623,NaN,RP11_469A15.2,-,RP11_469A15.2,MitoCarta3.0,literature,MIM,NaN,NA - newly define mitochondrial protein (forme...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [360]:
# IDs appearing in multiple rows. Watch for these later. Their rows will be collapsed into one. 
i = -1
for ids in df2['UniProt ID']:
    i += 1
    temp = list(df2['UniProt ID'][0:i].append(df2['UniProt ID'][(i+1):len(df2)]))
    if(ids in temp):
        print(ids)

P12532
P12532


In [361]:
print(df2['UniProt ID'][1135]=='NONE')

True


In [362]:
format = '.xml'; schema = xmlschema.XMLSchema('https://www.uniprot.org/docs/uniprot.xsd')
missing_ids = {}; Proteins = []; nameLengthDict = {}; initial = 0;i = initial
synList = []; notfound = []; notfoundDict = {}; deletethese = []; updatedIDs = []

for i in range(0,lendf2):
    previousPartOfDF2 = list(df2['UniProt ID'][0:i])
    if(len(df2['Synonyms'][i]) == 0):
        if(df2['UniProt ID'][i] != 'NONE'):
            allIDs, notfoundIDs, foundIDs, synList, sn = findURL(df2['UniProt ID'][i])
        else:
            allIDs = ['PROT_%i'%i]; notfoundIDs = ['PROT_%i'%i]; foundIDs = ['0']; synList = []; sn = 'N/A'; df['UniProt'][i] = 'PROT_%i'%i; df2['UniProt ID'][i] = 'PROT_%i'%i
        # If the protein is a human protein, proceed
        if((sn.lower() == 'homo sapiens' or sn.lower() == 'n/a')):
            j = -1
            # FIRST: If the ID had synonyms on UniProt, add them to the DF2 
            # If an ID is in multiple rows, add UniProt syns to one row
            # If an ID is not deprecated and in one row, add UniProt syns to row i
            # If an ID is deprecated and has one updated ID, add UniProt syns to row i
            # If an ID is depcreated and has multiple updated IDs, add UniProt syns to its separate rows at the end 
            for ids in foundIDs:
                if(foundIDs == ['0']): break
                j+=1

                ## Find the location of the ID in the DF2 ##
                if ids in previousPartOfDF2:
                    ind = previousPartOfDF2.index(ids) # find the part of the list it's in
                    df2['UniProt ID'][i] = 'REMOVE_DUPLICATE%i'%i
                    deletethese.append(i)
                else: # If the ID is not in the previous part of the DF2
                    try: # If ID is in DF2, find the location
                        ind = df2[df2['UniProt ID']==ids].index[0]
                    except: # If it's not, add it to DF2, find the location
                        df2 = df2.append({"UniProt ID": ids, "Synonyms": []}, ignore_index=True)
                        ind = df2[df2['UniProt ID']==ids].index[0]
                        
                ## Add synonyms to the proper location in the DF2 ##
                print(synList[j])
                for syn in synList[j]: df2['Synonyms'][ind].append(syn)
        
            # SECOND: Add synonyms from the original DF (MitoCarta)
            # (1) If an ID has one row, add MitoCarta synonyms. 
            # (2) If an ID has multiple rows, add MitoCarta to one row
            # (3) If an row has multiple IDs, don't add MitoCarta syns to any of its rows
            # (Maybe doesn't have case for if one of multiple updated IDs is a duplicate. Rare and also not part of this dataset currently)
            if(len(foundIDs) == 1 or foundIDs == ['0']): # (1) & (2)
                ids = allIDs[0]; print('Trying ', ids)
                if df2['UniProt ID'][i] != ids: updatedIDs.append(ids)
                if foundIDs == ['0']: df2['UniProt ID'][i] = 'PROT_%i'%i
                # Take synonyms from df, add to df2
                df_ind = i  
                if(ids in previousPartOfDF2):  df2_ind = previousPartOfDF2.index(ids)         # location to be added to
                else: df2_ind = i # If the ID is a duplicate and was previously used (i.e. already has a dedicated row)
                    
                # Add MitoCarta synonyms, description, and symbol. Ensure no duplicates within a protein
                df_syn = df['Synonyms'][df_ind].split('|'); df_syns = []; unroll(df_syn, df_syns, False); 
                for syn in df_syns: df2['Synonyms'][df2_ind].append(syn); 
                df2['Synonyms'][df2_ind].append(df['Description'][df_ind]); df2['Synonyms'][df2_ind].append(df['Symbol'][df_ind]); df2['Synonyms'][df2_ind] = list(set(df2['Synonyms'][df2_ind])); 
                print('Synonyms: ', df['Synonyms'][df_ind]); print('Description: ', df['Description'][df_ind]); print('Symbol: ', df['Symbol'][df_ind]); print('Final', i, df2['Synonyms'][df2_ind], '\n')
            
            elif('REMOVE_DUPLICATE' not in df2['UniProt ID'][i]): # (3)
                updatedIDs.append(df2['UniProt ID'][i])
                df2['UniProt ID'][i] = 'REPLACED_%i'%i
                deletethese.append(i)
                
        # If the protein is a nonhuman protein, it shouldn't be part of the list
        else:
            print(i, sn, df2['UniProt ID'][i], 'Delete')
            deletethese.append(i)
    
for i in deletethese:
    print('Removing', i)
    df2.drop(i,axis=0)
    
print('Updated IDs', updatedIDs)
print('Deleted IDs', deletethese)

1st Try
['Cytochrome c1, heme protein, mitochondrial', 'EC 7.1.1.8', 'Complex III subunit 4', 'Complex III subunit IV', 'Cytochrome b-c1 complex subunit 4', 'Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit', 'Cytochrome c-1']
Trying  P08574
Synonyms:  MC3DN6|UQCR4
Description:  cytochrome c1
Symbol:  CYC1
Final 0 ['Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit', 'Cytochrome c-1', 'cytochrome c1', 'Complex III subunit 4', 'UQCR4', 'MC3DN6', 'CYC1', 'EC 7.1.1.8', 'Cytochrome b-c1 complex subunit 4', 'Cytochrome c1, heme protein, mitochondrial', 'Complex III subunit IV'] 

1st Try
['Succinate dehydrogenase [ubiquinone] iron-sulfur subunit, mitochondrial', 'EC 1.3.5.1', 'Iron-sulfur subunit of complex II', 'Ip']
Trying  P21912
Synonyms:  CWS2|IP|PGL4|SDH|SDH1|SDH2|SDHIP
Description:  succinate dehydrogenase complex iron sulfur subunit B
Symbol:  SDHB
Final 1 ['SDHIP', 'SDH', 'SDH1', 'SDH2', 'SDHB', 'IP', 'CWS2', 'succinate dehydrogenase complex iron sulfur s

1st Try
['Pyruvate dehydrogenase E1 component subunit beta, mitochondrial', 'PDHE1-B', 'EC 1.2.4.1']
Trying  P11177
Synonyms:  PDHBD|PDHE1-B|PDHE1B|PHE1B
Description:  pyruvate dehydrogenase E1 subunit beta
Symbol:  PDHB
Final 16 ['PHE1B', 'PDHBD', 'PDHE1-B', 'EC 1.2.4.1', 'Pyruvate dehydrogenase E1 component subunit beta, mitochondrial', 'PDHB', 'pyruvate dehydrogenase E1 subunit beta', 'PDHE1B'] 

1st Try
['Cytochrome b-c1 complex subunit 2, mitochondrial', 'Complex III subunit 2', 'Core protein II', 'Ubiquinol-cytochrome-c reductase complex core protein 2']
Trying  P22695
Synonyms:  MC3DN5|QCR2|UQCR2
Description:  ubiquinol-cytochrome c reductase core protein 2
Symbol:  UQCRC2
Final 17 ['Cytochrome b-c1 complex subunit 2, mitochondrial', 'UQCR2', 'MC3DN5', 'Complex III subunit 2', 'ubiquinol-cytochrome c reductase core protein 2', 'Core protein II', 'Ubiquinol-cytochrome-c reductase complex core protein 2', 'QCR2', 'UQCRC2'] 

1st Try
['Succinate dehydrogenase [ubiquinone] cytochrom

1st Try
['Pyruvate dehydrogenase protein X component, mitochondrial', 'Dihydrolipoamide dehydrogenase-binding protein of pyruvate dehydrogenase complex', 'E3-binding protein', 'E3BP', 'Lipoyl-containing pyruvate dehydrogenase complex component X', 'proX']
Trying  O00330
Synonyms:  DLDBP|E3BP|OPDX|PDHXD|PDX1|proX
Description:  pyruvate dehydrogenase complex component X
Symbol:  PDHX
Final 31 ['OPDX', 'DLDBP', 'PDHX', 'PDHXD', 'proX', 'Dihydrolipoamide dehydrogenase-binding protein of pyruvate dehydrogenase complex', 'Lipoyl-containing pyruvate dehydrogenase complex component X', 'PDX1', 'pyruvate dehydrogenase complex component X', 'E3-binding protein', 'E3BP', 'Pyruvate dehydrogenase protein X component, mitochondrial'] 

1st Try
['Elongation factor G, mitochondrial', 'EF-Gmt', 'Elongation factor G 1, mitochondrial', 'mEF-G 1', 'Elongation factor G1', 'hEFG1']
Trying  Q96RP9
Synonyms:  COXPD1|EFG|EFG1|EFGM|EGF1|GFM|hEFG1|mtEF-G1
Description:  G elongation factor mitochondrial 1
Symbol:

1st Try
['Isocitrate dehydrogenase [NAD] subunit alpha, mitochondrial', 'EC 1.1.1.41', 'Isocitric dehydrogenase subunit alpha', 'NAD(+)-specific ICDH subunit alpha']
Trying  P50213
Synonyms:  -
Description:  isocitrate dehydrogenase (NAD(+)) 3 catalytic subunit alpha
Symbol:  IDH3A
Final 49 ['IDH3A', '-', 'Isocitrate dehydrogenase [NAD] subunit alpha, mitochondrial', 'Isocitric dehydrogenase subunit alpha', 'NAD(+)-specific ICDH subunit alpha', 'isocitrate dehydrogenase (NAD(+)) 3 catalytic subunit alpha', 'EC 1.1.1.41'] 

1st Try
['Cytochrome c oxidase assembly protein COX11, mitochondrial']
Trying  Q9Y6N1
Synonyms:  COX11P
Description:  cytochrome c oxidase copper chaperone COX11
Symbol:  COX11
Final 50 ['COX11', 'Cytochrome c oxidase assembly protein COX11, mitochondrial', 'COX11P', 'cytochrome c oxidase copper chaperone COX11'] 

1st Try
['Electron transfer flavoprotein-ubiquinone oxidoreductase, mitochondrial', 'ETF-QO', 'ETF-ubiquinone oxidoreductase', 'EC 1.5.5.1', 'Electron-tra

1st Try
['Carnitine O-palmitoyltransferase 2, mitochondrial', 'EC 2.3.1.21', 'Carnitine palmitoyltransferase II', 'CPT II']
Trying  P23786
Synonyms:  CPT1|CPTASE|IIAE4
Description:  carnitine palmitoyltransferase 2
Symbol:  CPT2
Final 67 ['EC 2.3.1.21', 'Carnitine palmitoyltransferase II', 'carnitine palmitoyltransferase 2', 'CPT II', 'IIAE4', 'CPT2', 'Carnitine O-palmitoyltransferase 2, mitochondrial', 'CPTASE', 'CPT1'] 

1st Try
['2-oxoisovalerate dehydrogenase subunit beta, mitochondrial', 'EC 1.2.4.4', 'Branched-chain alpha-keto acid dehydrogenase E1 component beta chain', 'BCKDE1B', 'BCKDH E1-beta']
Trying  P21953
Synonyms:  BCKDE1B|BCKDH E1-beta|E1B
Description:  branched chain keto acid dehydrogenase E1 subunit beta
Symbol:  BCKDHB
Final 68 ['BCKDH E1-beta', 'E1B', 'BCKDE1B', '2-oxoisovalerate dehydrogenase subunit beta, mitochondrial', 'Branched-chain alpha-keto acid dehydrogenase E1 component beta chain', 'branched chain keto acid dehydrogenase E1 subunit beta', 'BCKDHB', 'EC 

...

1st Try
['cDNA FLJ56092, highly similar to Pentatricopeptide repeat protein 1']
Trying  B4DJ38
Synonyms:  ATP5J2-PTCD1
Description:  ATP5MF-PTCD1 readthrough
Symbol:  ATP5MF-PTCD1
Final 1071 ['cDNA FLJ56092, highly similar to Pentatricopeptide repeat protein 1', 'ATP5MF-PTCD1 readthrough', 'ATP5J2-PTCD1', 'ATP5MF-PTCD1'] 

1st Try
['Caspase-8', 'CASP-8', 'EC 3.4.22.61', 'Apoptotic cysteine protease', 'Apoptotic protease Mch-5', 'CAP4', 'FADD-homologous ICE/ced-3-like protease', 'FADD-like ICE', 'FLICE', 'ICE-like apoptotic protease 5', 'MORT1-associated ced-3 homolog', 'MACH', 'Caspase-8 subunit p18', 'Caspase-8 subunit p10']
Trying  Q14790
Synonyms:  ALPS2B|CAP4|Casp-8|FLICE|MACH|MCH5
Description:  caspase 8
Symbol:  CASP8
Final 1072 ['MCH5', 'Caspase-8', 'MACH', 'ALPS2B', 'ICE-like apoptotic protease 5', 'Apoptotic cysteine protease', 'FLICE', 'EC 3.4.22.61', 'FADD-homologous ICE/ced-3-like protease', 'MORT1-associated ced-3 homolog', 'CAP4', 'Caspase-8 subunit p18', 'CASP8', 'Caspas

1st Try
['Mitoguardin 1', 'Protein FAM73A']
Trying  Q8NAN2
Synonyms:  FAM73A
Description:  mitoguardin 1
Symbol:  MIGA1
Final 1089 ['mitoguardin 1', 'FAM73A', 'Mitoguardin 1', 'Protein FAM73A', 'MIGA1'] 

1st Try
['Ribose-5-phosphate isomerase', 'EC 5.3.1.6', 'Phosphoriboisomerase']
Trying  P49247
Synonyms:  RPI|RPIAD
Description:  ribose 5-phosphate isomerase A
Symbol:  RPIA
Final 1090 ['Phosphoriboisomerase', 'RPIA', 'ribose 5-phosphate isomerase A', 'Ribose-5-phosphate isomerase', 'EC 5.3.1.6', 'RPIAD', 'RPI'] 

1st Try
['Bcl-2-binding component 3, isoforms 3/4', 'JFY-1', 'p53 up-regulated modulator of apoptosis']
Trying  Q96PG8
Synonyms:  JFY-1|JFY1|PUMA
Description:  BCL2 binding component 3
Symbol:  BBC3
Final 1091 ['JFY-1', 'PUMA', 'Bcl-2-binding component 3, isoforms 3/4', 'BBC3', 'JFY1', 'p53 up-regulated modulator of apoptosis', 'BCL2 binding component 3'] 

1st Try
['Bcl2-associated agonist of cell death', 'BAD', 'Bcl-2-binding component 6', 'Bcl-2-like protein 8', 'Bcl2-L-8

1st Try
['Adenylate cyclase type 10', 'EC 4.6.1.1', 'AH-related protein', 'Adenylate cyclase homolog', 'Germ cell soluble adenylyl cyclase', 'hsAC', 'sAC', 'Testicular soluble adenylyl cyclase']
Trying  Q96PN6
Synonyms:  HCA2|HEL-S-7a|SAC|SACI|Sacy|hsAC
Description:  adenylate cyclase 10
Symbol:  ADCY10
Final 1107 ['adenylate cyclase 10', 'sAC', 'EC 4.6.1.1', 'Adenylate cyclase homolog', 'SAC', 'Germ cell soluble adenylyl cyclase', 'HEL-S-7a', 'Sacy', 'Adenylate cyclase type 10', 'Testicular soluble adenylyl cyclase', 'ADCY10', 'SACI', 'AH-related protein', 'HCA2', 'hsAC'] 

1st Try
['Sphingosine kinase 2', 'SK 2', 'SPK 2', 'EC 2.7.1.91']
Trying  Q9NRA0
Synonyms:  SK 2|SK-2|SPK 2|SPK-2
Description:  sphingosine kinase 2
Symbol:  SPHK2
Final 1108 ['SK-2', 'SPHK2', 'EC 2.7.1.91', 'sphingosine kinase 2', 'SK 2', 'SPK 2', 'Sphingosine kinase 2', 'SPK-2'] 

1st Try
['tRNA (cytosine(34)-C(5))-methyltransferase, mitochondrial', 'EC 2.1.1.-', 'NOL1/NOP2/Sun domain family member 3']
Trying  Q9H

1st Try
['A-kinase anchor protein SPHKAP', 'SPHK1-interactor and AKAP domain-containing protein', 'Sphingosine kinase type 1-interacting protein']
Trying  Q2M3C7
Synonyms:  SKIP
Description:  SPHK1 interactor, AKAP domain containing
Symbol:  SPHKAP
Final 1130 ['Sphingosine kinase type 1-interacting protein', 'A-kinase anchor protein SPHKAP', 'SPHKAP', 'SPHK1-interactor and AKAP domain-containing protein', 'SKIP', 'SPHK1 interactor, AKAP domain containing'] 

1st Try
['N-acetylaspartate synthetase', 'NAA synthetase', 'EC 2.3.1.17', 'Camello-like protein 3', 'N-acetyltransferase 8-like protein']
Trying  Q8N9F0
Synonyms:  CML3|NACED|NAT8-LIKE
Description:  N-acetyltransferase 8 like
Symbol:  NAT8L
Final 1131 ['EC 2.3.1.17', 'CML3', 'NAA synthetase', 'NAT8-LIKE', 'NACED', 'Camello-like protein 3', 'N-acetyltransferase 8-like protein', 'N-acetylaspartate synthetase', 'N-acetyltransferase 8 like', 'NAT8L'] 

1st Try
['Mitochondrial coiled-coil domain protein 1']
Trying  P59942
Synonyms:  -
D

/home/ubuntu/anaconda3/envs/caseolap/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [363]:
deletethese

[119, 1069]

In [372]:
df2[117:121]

,UniProt ID,Synonyms
117,Q9NX18,[succinate dehydrogenase complex assembly fact...
118,Q96MF6,"[COQ10A, Coenzyme Q-binding protein COQ10 homo..."
119,Q99757,"[Thioredoxin, mitochondrial, TXN2, TXN, MT-TRX..."
120,Q9NVS2,"[39S ribosomal protein S18-3, mitochondrial, M..."


In [365]:
df2[473:476]

,UniProt ID,Synonyms
473,P16260,"[GDA, Graves disease carrier protein, hML7, So..."
474,Q96HJ9,[formation of mitochondrial complex V assembly...
475,Q53FZ2,"[Butyryl-coenzyme A synthetase 3, Protein SA h..."


In [366]:
df2[703:707]

,UniProt ID,Synonyms
703,P54868,[3-hydroxy-3-methylglutaryl coenzyme A synthas...
704,P51649,"[SSADH, EC 1.2.1.24, Succinate-semialdehyde de..."
705,P12532,"[CKMT1B, Ubiquitous mitochondrial creatine kin..."
706,Q9UG56,"[dJ858B16.2, Phosphatidylserine decarboxylase ..."


In [370]:
df2[1067:1071]

,UniProt ID,Synonyms
1067,Q9NRY6,"[phospholipid scramblase 3, -, PL scramblase 3..."
1068,O75438,"[NDUFB1, NADH:ubiquinone oxidoreductase subuni..."
1069,B4DJ38,"[cDNA FLJ56092, highly similar to Pentatricope..."
1070,Q14790,"[MCH5, Caspase-8, MACH, ALPS2B, ICE-like apopt..."


In [434]:
df2[352:354]

,UniProt ID,Synonyms
352,P27144,"[AK3L2, EC 2.7.4.6, Adenylate kinase 3-like, E..."
353,Q9NVI7,"[HAYOS, PHRINL, ATPase family AAA domain conta..."


In [439]:
df2[1007:1012]

,UniProt ID,Synonyms
1007,Q9BZ23,"[Pantothenate kinase 2, mitochondrial, Pantoth..."
1008,P35218,"[EC 4.2.1.1, CA-VA, CAVA, carbonic anhydrase 5..."
1009,Q9H857,"[EC 3.1.3.-, -, 5'-nucleotidase domain contain..."
1010,Q5T9A4,"[ATPase family AAA domain containing 3B, TOB3,..."
1011,O95900,"[EC 5.4.99.-, Mitochondrial mRNA pseudouridine..."


In [438]:
df2[1066:1074]

,UniProt ID,Synonyms
1066,O15091,"[Mitochondrial ribonuclease P protein 3, EC 3...."
1067,Q9NRY6,"[phospholipid scramblase 3, -, PL scramblase 3..."
1068,O75438,"[NDUFB1, NADH:ubiquinone oxidoreductase subuni..."
1069,B4DJ38,"[cDNA FLJ56092, highly similar to Pentatricope..."
1070,Q14790,"[MCH5, Caspase-8, MACH, ALPS2B, ICE-like apopt..."
1071,Q07817,"[BCLX, BCL2L, Bcl-X, Apoptosis regulator Bcl-X..."
1072,Q9NVH0,[Exonuclease 3'-5' domain-containing protein 2...
1073,Q8N2F6,"[ARMC10, PNAS112, PSEC0198, SVH, PNAS-112, Spl..."


In [435]:
df2[658:660]

,UniProt ID,Synonyms
658,O75891,[Cytosolic 10-formyltetrahydrofolate dehydroge...
659,O75127,"[PTCD1, -, Pentatricopeptide repeat-containing..."


In [371]:
df2.tail(5)

,UniProt ID,Synonyms
1131,A0A0B4J2F0,"[PIGBOS1, HP06981, PIGB opposite strand 1, PIG..."
1132,P86397,[3-hydroxyacyl-[acyl-carrier-protein] dehydrat...
1133,PROT_1135,"[RP11_469A15.2, -]"
1134,A0A0B4J2D5,[Glutamine amidotransferase-like class 1 domai...
1135,P0DPI2,[Glutamine amidotransferase-like class 1 domai...


In [369]:
# Drop rows without synonyms
i = -1; drop_rows = []
for row in df2.iterrows():
    i += 1
    if row[1][1] == []:
        print(i)
        drop_rows.append(i)
print(drop_rows)
for i in range(0,len(drop_rows)):
    j = len(drop_rows) - (i+1)
    df2 = df2.drop(index = drop_rows[j])

i = -1
df2.index = range(0,len(df2))
df2

119
1069
[119, 1069]


,UniProt ID,Synonyms
0,P08574,[Ubiquinol-cytochrome-c reductase complex cyto...
1,P21912,"[SDHIP, SDH, SDH1, SDH2, SDHB, IP, CWS2, succi..."
2,Q99807,"[COQ7, 5-demethoxyubiquinone hydroxylase, mito..."
3,P31040,"[PGL5, Flavoprotein subunit of complex II, CMD..."
4,P31930,"[Core protein I, UQCR1, Ubiquinol-cytochrome-c..."
...,...,...
1131,A0A0B4J2F0,"[PIGBOS1, HP06981, PIGB opposite strand 1, PIG..."
1132,P86397,[3-hydroxyacyl-[acyl-carrier-protein] dehydrat...
1133,PROT_1135,"[RP11_469A15.2, -]"
1134,A0A0B4J2D5,[Glutamine amidotransferase-like class 1 domai...


In [444]:
df2[1010:1011]

,UniProt ID,Synonyms
1010,Q5T9A4,"[ATPase family AAA domain containing 3B, TOB3,..."


### JSON OUTPUT: Protein ID:Name Dictionary 

In [445]:
df2.to_csv("MitoCarta.csv")
df2.to_json("MitoCarta.json")

### .TXT OUTPUT: Protein ID | Name,...,Name

In [3]:
#Read in the protein (ID: name, name, ... , name) JSON into a dictionary as it was before. 
Proteins = pd.read_json("MitoCarta.json")
proteinDict = {}
for row in Proteins.iterrows():
    proteinDict[row[1][0]] = row[1][1]
proteinDict

{'P08574': ['Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit',
  'Cytochrome c-1',
  'cytochrome c1',
  'Complex III subunit 4',
  'UQCR4',
  'MC3DN6',
  'CYC1',
  'EC 7.1.1.8',
  'Cytochrome b-c1 complex subunit 4',
  'Cytochrome c1, heme protein, mitochondrial',
  'Complex III subunit IV'],
 'P21912': ['SDHIP',
  'SDH',
  'SDH1',
  'SDH2',
  'SDHB',
  'IP',
  'CWS2',
  'succinate dehydrogenase complex iron sulfur subunit B',
  'Iron-sulfur subunit of complex II',
  'PGL4',
  'Succinate dehydrogenase [ubiquinone] iron-sulfur subunit, mitochondrial',
  'Ip',
  'EC 1.3.5.1'],
 'Q99807': ['COQ7',
  '5-demethoxyubiquinone hydroxylase, mitochondrial',
  'CLK-1',
  'DMQ hydroxylase',
  'Ubiquinone biosynthesis monooxygenase COQ7',
  'COQ10D8',
  'Timing protein clk-1 homolog',
  'CLK1',
  'CAT5',
  'EC 1.14.99.60',
  'coenzyme Q7, hydroxylase'],
 'P31040': ['PGL5',
  'Flavoprotein subunit of complex II',
  'CMD1GG',
  'FP',
  'Fp',
  'SDH1',
  'SDH2',
  'Succinate dehydrogena

In [8]:
# These were taken from and found in the other API. Look in the 'Filter the dictionary' section for most of them. Search for '.remove' to find them all
bad_names = ['Uncharacterized protein', '-', 'Mitochondrial', 'mitochondrial', 'IDE', 'Boo', 'ALS', 'LOW QUALITY PROTEIN', ]
for ID,names in proteinDict.items():
    newnames = []
    for name in names:
        if name not in bad_names and len(name) > 2:
            newnames.append(name)
    proteinDict[ID] = newnames
proteinDict

{'P08574': ['Ubiquinol-cytochrome-c reductase complex cytochrome c1 subunit',
  'Cytochrome c-1',
  'cytochrome c1',
  'Complex III subunit 4',
  'UQCR4',
  'MC3DN6',
  'CYC1',
  'EC 7.1.1.8',
  'Cytochrome b-c1 complex subunit 4',
  'Cytochrome c1, heme protein, mitochondrial',
  'Complex III subunit IV'],
 'P21912': ['SDHIP',
  'SDH',
  'SDH1',
  'SDH2',
  'SDHB',
  'CWS2',
  'succinate dehydrogenase complex iron sulfur subunit B',
  'Iron-sulfur subunit of complex II',
  'PGL4',
  'Succinate dehydrogenase [ubiquinone] iron-sulfur subunit, mitochondrial',
  'EC 1.3.5.1'],
 'Q99807': ['COQ7',
  '5-demethoxyubiquinone hydroxylase, mitochondrial',
  'CLK-1',
  'DMQ hydroxylase',
  'Ubiquinone biosynthesis monooxygenase COQ7',
  'COQ10D8',
  'Timing protein clk-1 homolog',
  'CLK1',
  'CAT5',
  'EC 1.14.99.60',
  'coenzyme Q7, hydroxylase'],
 'P31040': ['PGL5',
  'Flavoprotein subunit of complex II',
  'CMD1GG',
  'SDH1',
  'SDH2',
  'Succinate dehydrogenase [ubiquinone] flavoprotein sub

In [9]:
#output proteins to a txt file with the ID | name, name, ..., name format
f = open("MitoCartaAndHumanUPOnlyProteins.txt", "w")
for ID,names in proteinDict.items():
    line = ID + '|'
    lenNames = len(names)
    for i in range(0,lenNames):
        if(i < lenNames-1):
            line += names[i] + '|'
        else:
            line += names[i]
    print(line, file=f)
f.close()

In [429]:
for i in range(0,len(df2)):
    print(df2['UniProt ID'][i])

P08574
P21912
Q99807
P31040
P31930
Q5HYK3
P08559
O75208
P52815
P30049
P20674
Q86U28
O75439
P47985
P25705
Q02218
P11177
P22695
O14521
P82673
O14949
Q96EL3
P11182
Q16654
P40926
Q92552
O75390
Q9HAV7
P10515
P42704
P36957
O00330
Q96RP9
O95563
P28331
Q9H2W6
P56381
Q00325
Q9Y3D9
P07954
Q10713
P06576
Q5VUM1
Q9UDW1
Q9BUE6
Q9P2R7
Q9NZJ6
Q9NSE4
P82914
P50213
Q9Y6N1
Q16134
P62072
Q9BQ48
Q5T653
P12694
P07919
Q9BW72
P48047
P30084
Q6P1Q0
P12074
Q7KZN9
Q9Y4W6
P40939
P13804
O75251
P23786
P21953
O43837
Q15031
P16219
O95202
P56385
O60313
Q13825
P53597
P19404
Q9Y2Z9
Q8N983
Q8NFV4
P18859
O95169
P36776
P09622
O95831
Q96DC8
Q6UXV4
Q7Z7H8
Q96E11
O00217
P11310
Q16891
Q9Y5J7
P12235
Q9Y512
O75306
P49821
Q99798
Q8IYB8
P22830
P46199
P35232
Q6NVY1
Q9Y399
P38646
Q15526
P30048
Q9H3K2
Q8N442
Q9Y5L4
Q9HD34
Q9NX20
Q9NQ50
P51553
Q99643
O43674
Q9NX18
Q96MF6
Q99757
Q9NVS2
P09669
Q9Y6M9
Q9Y6C9
P56556
O43772
Q9BYD6
O43615
Q14061
Q5TGZ0
P04179
P14854
P21796
Q16740
Q16836
P28330
P24752
Q96EH3
O76031
O43181
Q9BYD3
Q07021
Q5JRX3